In [ ]:
!pip install git+https://github.com/huggingface/transformers@4970b23cedaf745f963779b4eae68da281e8c6ca

  Cloning https://github.com/huggingface/transformers (to revision 4970b23cedaf745f963779b4eae68da281e8c6ca) to /tmp/pip-req-build-66lqvao0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-66lqvao0
  Running command git rev-parse -q --verify 'sha^4970b23cedaf745f963779b4eae68da281e8c6ca'
  Running command git fetch -q https://github.com/huggingface/transformers 4970b23cedaf745f963779b4eae68da281e8c6ca
  Running command git checkout -q 4970b23cedaf745f963779b4eae68da281e8c6ca
  Resolved https://github.com/huggingface/transformers to commit 4970b23cedaf745f963779b4eae68da281e8c6ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.55.0.dev0-py3-none-any.whl size=12157751 sha256=71ef0c26f34d962952cfad6693c5f86562e99ae60049bb2bf142974fcfe254b0
  Stored in directory: /root/.cache/pip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv('/content/drive/MyDrive/program_summary_processed.csv')

In [ ]:
# İlk satırı seç
first_row = df.iloc[0]

# İlgili bilgileri al
title = first_row['title']
description = first_row['description']
level = first_row['level']
goal = first_row['goal']
equipment = first_row['equipment']

print(f"Deneme için kullanılacak başlık: {title}")
print(f"Deneme için kullanılacak açıklama: {description[:100]}...")
print(f"Deneme için kullanılacak seviye: {level}")
print(f"Deneme için kullanılacak hedef: {goal}")
print(f"Deneme için kullanılacak ekipman: {equipment}")

Deneme için kullanılacak başlık: (MASS MONSTER) High Intensity 4 Day Upper Lower Program
Deneme için kullanılacak açıklama: Build tones of muscular with this high intensity 4 day upper lower workout routine.

Weekly Setup:...
Deneme için kullanılacak seviye: ['Intermediate']
Deneme için kullanılacak hedef: ['Muscle & Sculpting', 'Bodyweight Fitness']
Deneme için kullanılacak ekipman: Full Gym


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import re

model_name_or_path = "tencent/Hunyuan-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")  # You may want to use bfloat16 and/or move to GPU here
messages = [
    {"role": "user", "content": "Write a short summary of the benefits of regular exercise"},
]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True,return_tensors="pt",
                                                enable_thinking=True # Toggle thinking mode (default: True)
                                                )

outputs = model.generate(tokenized_chat.to(model.device), max_new_tokens=2048)

output_text = tokenizer.decode(outputs[0])
print("output_text=",output_text)
think_pattern = r'<think>(.*?)</think>'
think_matches = re.findall(think_pattern, output_text, re.DOTALL)

answer_pattern = r'<answer>(.*?)</answer>'
answer_matches = re.findall(answer_pattern, output_text, re.DOTALL)

think_content = [match.strip() for match in think_matches][0]
answer_content = [match.strip() for match in answer_matches][0]
print(f"thinking_content:{think_content}\n\n")
print(f"answer_content:{answer_content}\n\n")


tokenizer_config.json: 0.00B [00:00, ?B/s]

The repository tencent/Hunyuan-7B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/tencent/Hunyuan-7B-Instruct .
 You can inspect the repository content at https://hf.co/tencent/Hunyuan-7B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_hy.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tencent/Hunyuan-7B-Instruct:
- tokenization_hy.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


hy.tiktoken: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

output_text= <|startoftext|>Write a short summary of the benefits of regular exercise<|extra_0|><think>
Okay, the user wants a short summary of the benefits of regular exercise. Let me start by recalling the main ones. First, physical health: cardiovascular benefits like better heart function and lower risk of diseases. Then mental health—stress reduction, improved mood, maybe even better sleep. Weight management is another big one; exercise burns calories and helps maintain a healthy weight. Bone health, since it strengthens muscles and bones, reducing osteoporosis risk. Immune system support, as regular activity can boost the body's defenses. Maybe also energy levels and flexibility. I should make sure to keep it concise, not too technical. Let me organize these points smoothly. Start with physical health, then mental, weight, bone, immune, and energy. Check if I'm missing anything important. Oh, maybe mention long-term benefits like longevity. Yeah, that's good. Need to keep each po

In [ ]:
import re
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import tqdm

# 0️⃣ DataFrame'inizi yükleyin
# df = pd.read_csv("your_input.csv")

# 1️⃣ Tokenizer & model (7B Instruct)
model_name = "tencent/Hunyuan-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model     = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")

all_schedules = []

# 2️⃣ Döngüyü tqdm ile sararak ilerlemeyi göster
for row in tqdm.tqdm(df.itertuples(index=False), total=len(df), desc="Antrenman programları"):
    title, goal, level, equipment = row.title, row.goal, row.level, row.equipment

    messages = [
        {"role": "system", "content":
            "You are concise. "
            "List only the 7 workout days with a one-line title each. "
            "Do not add explanations or extra text."
        },
        {"role": "user", "content":
            "Provide 7 lines for a week-long schedule based on:\n"
            f"Title: {title}\n"
            f"Goal: {goal}\n"
            f"Level: {level}\n"
            f"Equipment: {equipment}"
        }
    ]

    # ← Tek bir Tensor dönüyor
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=True
    ).to(model.device)

    # Modelden üret
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=512,
        temperature=0.3,
        pad_token_id=tokenizer.eos_token_id
    )

    # Yanıtı decode et ve <answer> bloğunu al
    full = tokenizer.decode(outputs[0], skip_special_tokens=False)
    answer_block = re.search(r"<answer>(.*?)</answer>", full, re.DOTALL)
    text = answer_block.group(1).strip() if answer_block else full

    # “Day 1:” … “Day 7:” ayrıştırması
    days    = [f"Day {i}:" for i in range(1, 8)]
    indices = {d: text.find(d) for d in days}

    snippets = {"Title": title}
    for i, day in enumerate(days):
        start = indices[day]
        if start == -1:
            snippets[day] = ""
        else:
            start += len(day)
            next_idx = indices.get(days[i+1], -1) if i < 6 else -1
            end = next_idx if (i < 6 and next_idx != -1) else len(text)
            snippets[day] = text[start:end].strip().split("\n")[0][:750]

    all_schedules.append(snippets)

# 3️⃣ Sonuçları DataFrame’e yaz ve CSV’ye kaydet
df_all = pd.DataFrame(all_schedules)
df_all.columns = [c.replace(" ", "_").replace(":", "") for c in df_all.columns]
df_all.to_csv("parcali_programs.csv", index=False, encoding="utf-8-sig")

print("✅ 'parcali_programs.csv' oluşturuldu. Sütunlar:", df_all.columns.tolist())

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_hy.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tencent/Hunyuan-7B-Instruct:
- tokenization_hy.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


hy.tiktoken: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

Antrenman programları: 100%|██████████| 2598/2598 [16:08:54<00:00, 22.38s/it]

✅ 'parcali_programs.csv' oluşturuldu. Sütunlar: ['Title', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7']
